In [1]:
import requests
import datetime
import re

In [8]:
def getApiKey(appName):
    with open(f'{appName}Key.txt') as keyFile:
        return next(keyFile)
    
def music_handler(f, o):
    current_line = next(f)
    if current_line[0] == "!":
        return current_line 
    
    trackLink = current_line.strip()
    parameters = {"url": trackLink, "format": "json"}
    website = re.search('\/\/(.+?)\/', trackLink).group(1)
    trackJson = requests.get(f'https://{website}/oembed', params = parameters).json()
    trackThumb = trackJson['thumbnail_url']
    trackTitle = trackJson['title']
    htmlBlock = f'<a href="{trackLink}"><div class="ach"> <img class="ach-img" src={trackThumb}></img>{trackTitle}</div></a>\n'
    o.write(htmlBlock)
    music_handler(f, o)
    
def playlist_handler(f, o):
    current_line = next(f)
    if current_line[0] == "!":
        return current_line 
    
    trackLink = current_line.strip()
    parameters = {"url": trackLink, "api_key": getApiKey("iframely")}
    trackJson = requests.get(f'https://iframe.ly/api/oembed', params = parameters).json()
    print(trackJson)
    trackThumb = trackJson['thumbnail_url']
    trackTitle = trackJson['title']
    htmlBlock = f'<a href="{trackLink}"><div class="ach"> <img class="ach-img" src={trackThumb}></img>{trackTitle}</div></a>\n'
    o.write(htmlBlock)
    playlist_handler(f, o)
    
with open("data.txt", "r+") as f:
    output = open(f'output {str(datetime.datetime.now())}.txt', "x")
    line = next(f)
    #check for a playlist link under the playlist flag
    if(line == "!playlist\n"):
        line = playlist_handler(f, output)
    # check for repeated music links under music flags and projects under project flag.
    while line != "":
        if line == "!music\n":
            line = music_handler(f, output)
            continue # so that we can check for EOF after recursion
        # elif l == "!projects":
        line = ""
    output.close()
            
    
        

{'url': 'https://music.apple.com/us/playlist/quinn-top-100-2021/pl.u-4JomajBFaeKMDl4', 'type': 'rich', 'version': '1.0', 'title': 'quinn top 100 2021 by Quinn Collins', 'provider_name': 'Apple\xa0Music', 'description': 'Playlist · 100 Songs', 'thumbnail_url': 'https://is4-ssl.mzstatic.com/image/thumb/5LKnBrQIUARZ6-nbj9eaGw/1200x630wp-60.jpg', 'thumbnail_width': 1200, 'thumbnail_height': 630, 'height': 450, 'html': '<div style="left: 0; width: 100%; height: 450px; position: relative;"><iframe src="https://embed.music.apple.com/us/playlist/quinn-top-100-2021/pl.u-4JomajBFaeKMDl4" style="top: 0; left: 0; width: 100%; height: 100%; position: absolute; border: 0;" allowfullscreen allow="encrypted-media;"></iframe></div>', 'messages': ['You may also change height by appending ?_height= to the URL itself.'], 'cache_age': 86400, 'options': {'_height': {'label': 'Adjust height', 'value': 450, 'placeholder': 'ex.: 600, in px'}}}
